In [1]:
from sklearn.datasets import make_multilabel_classification
# from bls2 import broadnet
import bls1,bls3
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from skmultilearn.dataset import load_dataset
from skmultilearn.dataset import available_data_sets
from sklearn.metrics import accuracy_score,hamming_loss,f1_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from random import seed
from random import randrange,random
from csv import reader

D:\anaconda3\envs\openne\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
set([x[0] for x in available_data_sets().keys()])

{'Corel5k',
 'bibtex',
 'birds',
 'delicious',
 'emotions',
 'enron',
 'genbase',
 'mediamill',
 'medical',
 'rcv1subset1',
 'rcv1subset2',
 'rcv1subset3',
 'rcv1subset4',
 'rcv1subset5',
 'scene',
 'tmc2007_500',
 'yeast'}

In [3]:
X_train, y_train, feature_names, label_names = load_dataset('yeast', 'train')
X_test, y_test, _, _ = load_dataset('yeast', 'test')
X=np.vstack([X_train.toarray(),X_test.toarray()])
y=np.vstack([y_train.toarray(),y_test.toarray()])

yeast:train - exists, not redownloading
yeast:test - exists, not redownloading


In [4]:
def subsample(dataset, labelset,ratio=0.6):
    n_sample = round(dataset.shape[0] * ratio)
    sample = np.ndarray((n_sample,dataset.shape[1]))
    label=np.ndarray((n_sample,labelset.shape[1]))
    for i in range(n_sample):
        index = randrange(dataset.shape[0])
        sample[i]=dataset[index]
        label[i]=labelset[index]
    return sample,label

In [5]:
sample,label=subsample(X,y)

In [6]:
bls = bls3.broadnet(maptimes = 30,
               enhencetimes = 40,
               map_function = 'relu',
               enhence_function = 'sigmoid',
               batchsize = 100,
               reg = 0.001)

In [7]:
bls.fit(sample,label)

number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 30.12475 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 


In [8]:
bls.alpha

[0.6499999999999997,
 0.6899999999999997,
 0.8399999999999996,
 0.18999999999999995,
 0.7099999999999996,
 0.8399999999999996,
 0.6499999999999997,
 0.3899999999999999,
 0.48999999999999977,
 0.7499999999999997,
 0.5299999999999998,
 0.21999999999999995,
 0.1,
 0.2699999999999999]

In [9]:
pred=[]
for i in range(10):
    sample,label=subsample(X_train.toarray(),y_train.toarray())
    bls = bls3.broadnet(maptimes = 30,
               enhencetimes = 40,
               map_function = 'relu',
               enhence_function = 'sigmoid',
               batchsize = 100,
               reg = 0.001)
    bls.fit(sample,label)
    pred.append(bls.predict(X_test))

number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 26.51949 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.86175 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.55748 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.6844 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.49324 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.96667 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 29.33297 minvalue 0.0 
enhence nodes maxval

In [10]:
type(pred[1])

numpy.matrix

In [11]:
pred_sum=pred[0]
for i in range(1,len(pred)):
    pred_sum+=pred[i]
pred_sum

matrix([[ 1.,  1.,  0., ...,  9., 10.,  0.],
        [ 0.,  1.,  8., ...,  5.,  7.,  0.],
        [ 4.,  9.,  3., ...,  7.,  6.,  0.],
        ...,
        [ 3.,  4.,  8., ...,  4.,  5.,  0.],
        [ 2.,  7.,  6., ..., 10., 10.,  0.],
        [10.,  6.,  6., ...,  9.,  9.,  0.]])

In [12]:
pred_sum[pred_sum<5]=0
pred_sum[pred_sum>=5]=1
pred_sum

matrix([[0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 1., ..., 1., 1., 0.],
        [0., 1., 0., ..., 1., 1., 0.],
        ...,
        [0., 0., 1., ..., 0., 1., 0.],
        [0., 1., 1., ..., 1., 1., 0.],
        [1., 1., 1., ..., 1., 1., 0.]])

In [13]:
accuracy_score(y_pred=pred_sum,y_true=y_test.toarray())

0.17011995637949837

In [14]:
dtc=DecisionTreeClassifier()
dtc.fit(X_train.toarray(),y_train.toarray())
dtc.score(X_test.toarray(),y_test.toarray())

0.11886586695747

In [15]:
f1_score(y_pred=pred_sum,y_true=y_test,average='micro')

0.6563106796116506

In [16]:
from multiprocessing.dummy import Pool

In [17]:
predition=[0 for i in range(10)]
predition

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
def bagging_net(i):
    bls = bls3.broadnet(maptimes = 30,
               enhencetimes = 40,
               map_function = 'relu',
               enhence_function = 'sigmoid',
               batchsize = 100,
               reg = 0.001)
    sample,label=subsample(X_train.toarray(),y_train.toarray())
    bls.fit(sample,label)
    pred=bls.predict(X_test.toarray())
    predition[i]=pred

In [19]:
bagging_net(1)

number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.64543 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 


In [20]:
items=[i for i in range(10)]
p=Pool(10)
p.map(bagging_net,items)
p.close()
p.join()

number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 26.80949 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 26.64694 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 26.66545 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000enhence nodes maxvalue 1.0 minvalue 0.0 

mapping nodes maxvalue 28.80375 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.60255 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.49997 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.72365 minvalue 0.0 
enhence nodes maxva

In [21]:
predition

[matrix([[0., 0., 0., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 0., 0., ..., 1., 1., 0.],
         ...,
         [1., 1., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 1., 1., 0.],
         [1., 1., 0., ..., 1., 1., 0.]]),
 matrix([[0., 1., 1., ..., 1., 1., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 1., 1., 0.],
         ...,
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [1., 1., 1., ..., 1., 1., 0.]]),
 matrix([[0., 0., 0., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         ...,
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.]]),
 matrix([[0., 0., 0., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         ...,
         [0., 1., 1., ..., 0., 0., 0.],
         [0., 1., 1., ..., 1., 1., 0.],
         [0., 1., 

In [22]:
import timeit
items=[i for i in range(20)]
def bagging_net(i):
    bls = bls3.broadnet(maptimes = 30,
               enhencetimes = 40,
               map_function = 'relu',
               enhence_function = 'sigmoid',
               batchsize = 100,
               reg = 0.001)
    sample,label=subsample(X_train.toarray(),y_train.toarray())
    bls.fit(sample,label)
    pred=bls.predict(X_test.toarray())
    return pred
start = timeit.default_timer()
p=Pool(8)
pre=p.map(bagging_net,items)
p.close()
p.join()
end = timeit.default_timer()
print('the processing time:', str(end-start), 's')

number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.57107 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.3153 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 32.02731 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.66838 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.11845 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 32.55351 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 30.08235 minvalue 0.0 
enhence nodes maxval

In [23]:
start = timeit.default_timer()
pred=[]
for i in range(10):
    sample,label=subsample(X_train.toarray(),y_train.toarray())
    bls = bls3.broadnet(maptimes = 30,
               enhencetimes = 40,
               map_function = 'relu',
               enhence_function = 'sigmoid',
               batchsize = 100,
               reg = 0.001)
    bls.fit(sample,label)
    pred.append(bls.predict(X_test))
end = timeit.default_timer()
print('the processing time:', str(end-start), 's')

number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 26.93595 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 26.65798 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.39671 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.33391 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.32675 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 27.06594 minvalue 0.0 
enhence nodes maxvalue 1.0 minvalue 0.0 
number of mapping nodes 3000, number of enhence nodes 4000
mapping nodes maxvalue 28.68762 minvalue 0.0 
enhence nodes maxva

In [24]:
pre[0]

matrix([[0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 1., 0.]])

In [25]:
pre_sum=pred[0]
for i in range(1,len(pre)):
    pre_sum+=pre[i]
pre_sum

matrix([[ 1.,  2.,  5., ..., 19., 20.,  0.],
        [ 0.,  4., 18., ..., 16., 16.,  0.],
        [ 7.,  6.,  4., ..., 16., 16.,  0.],
        ...,
        [ 2.,  6., 14., ..., 11.,  9.,  2.],
        [ 2., 10., 14., ..., 18., 19.,  0.],
        [11.,  4., 10., ..., 19., 18.,  0.]])

In [26]:
pre_sum[pre_sum<5]=0
pre_sum[pre_sum>=5]=1
print(accuracy_score(y_pred=pre_sum,y_true=y_test))
print(hamming_loss(y_pred=pre_sum,y_true=y_test))
print(f1_score(y_pred=pre_sum,y_true=y_test,average='micro'))

0.10141766630316248
0.2520641844524069
0.6521926053310405


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rfc=RandomForestClassifier(n_estimators=20)
rfc.fit(X_train.toarray(),y_train.toarray())


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [29]:
rfc.score(X_test,y_test)

0.13086150490730644